In [1]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 7.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4b8692873bef9873520866649f8a74647f747e6b6903ba371132b2291836c63c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=9a47e30f3370b2f07ad884ef13ad6d2a5f9942f2715b8e8d326d573665273511
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252b

In [2]:
import pandas as pd 

import joblib

from warnings import filterwarnings
filterwarnings("ignore")

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

import transformers
from transformers import BertTokenizer, TFBertModel
from simpletransformers.classification import ClassificationModel

2023-01-24 07:06:23.163862: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-munged-data/amerix_sentiment_analysis_preprocessed_data.csv', index_col=0)

In [4]:
df.head(10)

,tweet_created_at,text,Sentiment
0,2022-12-30 22:37:46,men refused listen toattend amerix classes men...,0
1,2022-12-30 22:30:58,ovobrezzzy take amerix next,2
2,2022-12-30 22:18:00,ayodeleyo amerix nah i m good masculinity heal...,1
3,2022-12-30 20:42:54,amerixdontsimp there need dating first place,0
4,2022-12-30 20:32:23,dyeanna amerix kibe from cos nothing changing ...,1
5,2022-12-30 20:25:56,salimthedon mwendiajnr i see unblocked me one ...,0
6,2022-12-30 20:22:24,germankenyan mlphilosophy yes also cook i woul...,0
7,2022-12-30 20:10:49,padremusa amerix said alcohol menace wrecking ...,0
8,2022-12-30 19:36:49,dodopicard youre eastafrica probably knows amerix,2
9,2022-12-30 19:30:58,hbkactivist amerix kibe heal,2


In [5]:
x = df['text']
y = df['Sentiment']

In [6]:
x.head()

0    men refused listen toattend amerix classes men...
1                          ovobrezzzy take amerix next
2    ayodeleyo amerix nah i m good masculinity heal...
3         amerixdontsimp there need dating first place
4    dyeanna amerix kibe from cos nothing changing ...
Name: text, dtype: object

In [7]:
y.head()

0    0
1    2
2    1
3    0
4    1
Name: Sentiment, dtype: int64

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, train_size = .75)

In [9]:
model_bert_large=ClassificationModel('bert', 
                                     'bert-large-uncased', 
                                     num_labels=2, 
                                     args={'max_seq_length':128, 
                                           'train_batch_size': 16,
                                           'reprocess_input_data': True, 
                                           'overwrite_output_dir': True,
                                           'num_train_epochs': 2},
                                     use_cuda=False) 

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]